In [13]:
import requests
import json
import pandas as pd
import statistics
import os
import pickle

# Get league data

To get league ID: Fantasy -> Leagues and Cups -> select your league

league_id = numbers before "/standings/" in the URL

ie: for my league, the URL is https://fantasy.premierleague.com/leagues/736795/standings/c, so my league ID would be "780695".

In [14]:
def save_league_data(league_id):
    '''
    Function to save Fantasy Premier League data for a specific league.
    
    Parameters:
    - league_id: The ID of the league to fetch data for.
    
    The function fetches the league data from the FPL API and saves it as a JSON file in a directory named "league_<league_id>".
    '''
        
    league_id = str(league_id)

    # create main and save_dir
    main_dir = f"league_{league_id}"
    if not os.path.exists(main_dir):
        os.makedirs(main_dir)

    save_dir = f"league_{league_id}/league_data"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # fetch league data from API using this URL
    # to get league ID: Fantasy -> Leagues and Cups -> {your_league}...
    # league_id = numbers before "/standings/" in the URL
    url = f"https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings/"
    
    # send a GET request to the URL
    response = requests.get(url)
    
    # check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # set the filename with league_id 
        file_name = f"{save_dir}/league_{league_id}_data.json"
        
        # Write the JSON data to a file
        with open(file_name, 'w') as file:
            json.dump(data, file, indent=4)
        
        print(f"Data saved successfully in {file_name}")
    else:
        print(f"Failed to fetch data: HTTP {response.status_code}")

This function will make a new folder for all of your league's data! The name of this folder will be *league_{league_id}*. All subsequent data and output will be saved to this folder. 

In [15]:
# Example usage
league_id = '780695' ## CHANGE ME to your league's ID 
save_league_data(league_id)

Data saved successfully in league_780695/league_data/league_780695_data.json


# Get user data from league

In [16]:
def get_users(league_id):
    f = open(f'league_{league_id}/league_data/league_{league_id}_data.json')
    
    # returns JSON object as a dictionary
    data = json.load(f)

    # Extract results
    # results = data['new_entries']['results'] # for start of league
    results = data['standings']['results'] # after the 1st GW

    # List of user IDs
    user_ids = [result['entry'] for result in results]
    print("List of user_ids:", user_ids)

    # Dictionary mapping entry to player name
    # entry_to_player = {result['entry']: result['player_first_name'] +" "+ result['player_last_name'] for result in results} # b4/during 1st GW
    entry_to_player = {result['entry']: result['player_name'] for result in results} # after the 1st GW
    print("Entry to Player Name Mapping:", entry_to_player)

    # Specify the file path
    pickle_file_path = f"league_{league_id}/league_data/league_{league_id}_entry_to_player_dict.pickle"

    # Save the dictionary as a pickle file
    with open(pickle_file_path, 'wb') as file:
        pickle.dump(entry_to_player, file)

    print(f"Entry to Player Mapping dictionary saved successfully as {pickle_file_path}")


This function will save a dictionary mapping of user name -> user ID, in pickle format. 

In [17]:
# Example usage
get_users(league_id)

List of user_ids: [3736356, 7296084, 5716768, 4040748, 5534893, 3771330, 2073867, 2581654, 3299662, 3321209, 3190565, 754678, 6190785, 5682456, 3950036, 8439351, 1104743, 1662216, 6184318, 7263639, 4101297]
Entry to Player Name Mapping: {3736356: 'Brady Gardner', 7296084: 'Shaan Nerurkar', 5716768: 'Marc Mays', 4040748: 'Brian Gardner', 5534893: 'Cole Mays', 3771330: 'Seamus Meagher', 2073867: 'Ethan Wilt', 2581654: 'Shubh Parekh', 3299662: 'Luka Pappas', 3321209: 'Cam Morse', 3190565: 'Harry Rohlfing', 754678: 'Gabe Fajardo', 6190785: 'Harrison Savage', 5682456: 'Aidan Trip', 3950036: 'Jay Mehta', 8439351: 'Ben Brennan', 1104743: 'William Sedgley', 1662216: 'tom sedgley', 6184318: 'Jacques Hughes', 7263639: 'Jehan Mody', 4101297: 'Matt Buller'}
Entry to Player Mapping dictionary saved successfully as league_780695/league_data/league_780695_entry_to_player_dict.pickle


# Get GW history for each user in the league (standings across time)

In [18]:
def fetch_and_save_history(entry_to_player_mapping):

    # create the save directory
    save_dir = f"league_{league_id}/user_data"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # fetch the history data for each player
    for entry, player_name in entry_to_player_mapping.items():
        url = f"https://fantasy.premierleague.com/api/entry/{entry}/history/"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            file_name = f"{save_dir}/{player_name.replace(' ', '_')}_{entry}_history.json"
            with open(file_name, 'w') as file:
                json.dump(data, file, indent=4)
            print(f"Data saved for {player_name} as {file_name}")
        else:
            print(f"FAILED to fetch data for {player_name} (entry {entry}), \nstatus code: {response.status_code}")

def get_gw_history_df(league_id, entry_to_player_mapping, save_dir=f'league_{league_id}/league_data/league_{league_id}_gw_history_df.csv'):
    user_data_dir = f"league_{league_id}/user_data"

    # first, get the number of current gameweeks
    N_GWs = []

    for entry, player_name in entry_to_player_mapping.items():
        file_name = f"{user_data_dir}/{player_name.replace(' ', '_')}_{entry}_history.json"
        with open(file_name) as file:
            data = json.load(file)
        history = data['current']
        N_GWs.append(len(history))

    # get the total number of current GWs (using mode, or can do max)
    N_GWs = statistics.mode(N_GWs)

    # create blank df with gws
    gws = [f'GW{gw}' for gw in range(1,N_GWs+1)]

    gw_history_df = pd.DataFrame(columns=['GW'])
    gw_history_df['GW'] = gws

    # create the GW scoring history table
    for entry, player_name in entry_to_player_mapping.items():
        file_name = f"{user_data_dir}/{player_name.replace(' ', '_')}_{entry}_history.json"
        with open(file_name) as file:
            data = json.load(file)

        history = data['current']

        cum_points = 0
        tot_points = []
        for game in history:
            points = game['points']
            cum_points += points
            tot_points.append(cum_points)
        
        # if a player joined late... change their name + ooints for select GWs
        # Jacques missed GW 1! ... adjust for num points he got (84)
        # if player_name == 'Jacques Hughes':
        #     tot_points.insert(0, 84)
        
        # add cumulative sum of GWs for each player to df
        # this is the format for the bar_chart_race!
        gw_history_df[player_name] = tot_points
    
    # save the df
    gw_history_df.to_csv(save_dir, index=False)

The ```fetch_and_save_history``` function will save each individual users data across time to the *league_{league_id}/user_data* folder. 

In [19]:
# load the Entry to Player Mapping dictionary
with open(rf"league_{league_id}/league_data/league_{league_id}_entry_to_player_dict.pickle", "rb") as entry_to_player_mapping:
    entry_to_player_mapping = pickle.load(entry_to_player_mapping)

fetch_and_save_history(entry_to_player_mapping) 

Data saved for Brady Gardner as league_780695/user_data/Brady_Gardner_3736356_history.json
Data saved for Shaan Nerurkar as league_780695/user_data/Shaan_Nerurkar_7296084_history.json
Data saved for Marc Mays as league_780695/user_data/Marc_Mays_5716768_history.json
Data saved for Brian Gardner as league_780695/user_data/Brian_Gardner_4040748_history.json
Data saved for Cole Mays as league_780695/user_data/Cole_Mays_5534893_history.json
Data saved for Seamus Meagher as league_780695/user_data/Seamus_Meagher_3771330_history.json
Data saved for Ethan Wilt as league_780695/user_data/Ethan_Wilt_2073867_history.json
Data saved for Shubh Parekh as league_780695/user_data/Shubh_Parekh_2581654_history.json
Data saved for Luka Pappas as league_780695/user_data/Luka_Pappas_3299662_history.json
Data saved for Cam Morse as league_780695/user_data/Cam_Morse_3321209_history.json
Data saved for Harry Rohlfing as league_780695/user_data/Harry_Rohlfing_3190565_history.json
Data saved for Gabe Fajardo a

The ```get_gw_history_df``` function will make a df of this user history data. It will be saved to the *league_{league_id}* folder. 

In [20]:
get_gw_history_df(league_id, entry_to_player_mapping)